# EDA Seguro Médico

Notebook de análisis exploratorio de datos (EDA) sobre el dataset de costos de seguros médicos.
Inspirado en el formato y explicaciones de Proyecto_Sueno.

## 1. Carga de librerías y datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split

DATA_DIR = 'data'
OUTPUT_DIR = 'output'
FIGS_DIR = os.path.join(OUTPUT_DIR, 'figs')
os.makedirs(FIGS_DIR, exist_ok=True)

# Carga de datos
df = pd.read_csv(os.path.join(DATA_DIR, 'insurance.csv'))
df.head()

## 2. Análisis inicial y descripción del conjunto de datos
- Revisamos tipos de variables
- Exploramos datos faltantes
- Describimos numéricas
- Justificamos cada paso

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe(include='all')

> **Justificación**: Revisar tipos y valores faltantes asegura que el análisis sea confiable y permite decidir si se requiere imputación o limpieza adicional.

## 3. Transformaciones iniciales
- Convertimos columnas categóricas a tipo 'category'.
- Aclaramos por qué lo hacemos: facilita visualizaciones y tratamiento posterior.

In [ ]:
cat_cols = ['sex', 'smoker', 'region']
for c in cat_cols:
    df[c] = df[c].astype('category')

## 4. Análisis univariante
- Revisamos distribuciones de numéricas y categóricas individualmente.
- Incluimos deducciones tipo: "Observamos que la mayoría de los pacientes no son fumadores, pero los fumadores tienen costos mucho más altos".

In [ ]:
# Variables numéricas
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in num_cols:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribución de {col}')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'univariate_{col}.png'))
    plt.show()

In [ ]:
# Variables categóricas + children (discreta)
for col in cat_cols + ['children']:
    plt.figure(figsize=(6,4))
    sns.countplot(data=df, x=col)
    plt.title(f'Conteo por {col}')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'univariate_{col}.png'))
    plt.show()

## Deducciones univariantes
- Escribe aquí tus observaciones sobre cada variable, por ejemplo:
  - La variable `smoker` tiene una minoría de casos positivos, pero es crucial analizar su impacto en el costo.
  - El costo (`charges`) está sesgado a la derecha, sugiriendo algunos pacientes con costos extremadamente altos.

## 5. Filtrado de outliers
- Aplicamos método IQR sobre `charges` para quitar extremos y facilitar el análisis.

In [ ]:
def remove_outliers_iqr(df_in, col, k=1.5):
    q1 = df_in[col].quantile(0.25)
    q3 = df_in[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - k * iqr
    upper = q3 + k * iqr
    mask = (df_in[col] >= lower) & (df_in[col] <= upper)
    return df_in.loc[mask]

df_filtered = remove_outliers_iqr(df, 'charges', k=1.5)
df_filtered.shape

> **Justificación**: El corte por IQR permite trabajar con datos más representativos y menos sesgados por casos extremos.

## 6. Tratamiento de la variable objetivo
- Decidimos si transformar `charges` (log) o dejarla como está.

In [ ]:
if (df_filtered['charges'] <= 0).any():
    print('Advertencia: charges tiene valores no positivos; no se puede aplicar log sin ajuste')
else:
    df_filtered['log_charges'] = np.log(df_filtered['charges'])

> **Justificación**: El log ayuda a normalizar la variable objetivo si queremos modelos lineales o entender mejor la dispersión.

## 7. Análisis bivariante
- Relación entre `charges` y cada variable (gráficas y deducciones).

In [ ]:
# Scatter para numéricas vs charges
for col in num_cols:
    if col == 'charges' or col == 'log_charges':
        continue
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=df_filtered, x=col, y='charges', alpha=0.6)
    plt.title(f'Charges vs {col}')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'bivariate_charges_vs_{col}.png'))
    plt.show()

In [ ]:
# Boxplot para categóricas vs charges
for col in cat_cols + ['children']:
    plt.figure(figsize=(8,5))
    sns.boxplot(data=df_filtered, x=col, y='charges')
    plt.title(f'Charges por {col}')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'bivariate_charges_by_{col}.png'))
    plt.show()

## 8. Matriz de correlación y filtrado de variables redundantes
- Mostramos correlaciones con `charges`.
- Eliminamos variables redundantes si corresponde.

In [ ]:
corr = df_filtered.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', square=True)
plt.title('Matriz de correlación (numéricas)')
plt.tight_layout()
plt.savefig(os.path.join(FIGS_DIR, 'correlation_matrix.png'))
plt.show()
corr['charges'].sort_values(ascending=False)

## 9. División de dataset en `train.csv` y `test.csv` (80/20, estratificado)
- Estratificamos por bins de `charges` para mantener la proporción.

In [ ]:
n_bins = 5
df_filtered['charges_bin'] = pd.qcut(df_filtered['charges'], q=n_bins, labels=False, duplicates='drop')
train_df, test_df = train_test_split(df_filtered, test_size=0.2, random_state=42, stratify=df_filtered['charges_bin'])
prop_train = train_df['charges_bin'].value_counts(normalize=True).sort_index()
prop_test = test_df['charges_bin'].value_counts(normalize=True).sort_index()
print('Proporciones por bin en train:', prop_train)
print('Proporciones por bin en test:', prop_test)
train_df = train_df.drop(columns=['charges_bin'])
test_df = test_df.drop(columns=['charges_bin'])
train_df.to_csv(os.path.join(DATA_DIR, 'train.csv'), index=False)
test_df.to_csv(os.path.join(DATA_DIR, 'test.csv'), index=False)

> **Justificación**: Estratificar los splits hace que los modelos se evalúen en test sobre la misma distribución que el train.

## 10. Guardado y resumen final
- Guardamos deducciones, correlaciones y resumen en archivos de texto.
- Explicamos cómo usar los resultados para modelado futuro.

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
with open(os.path.join(OUTPUT_DIR, 'deductions.txt'), 'w', encoding='utf-8') as f:
    f.write('Deducciones:\n')
    f.write('- charges es la variable objetivo.\n')
    f.write('- Fumar y BMI tienen alto impacto en el costo.\n')
    f.write('- Los outliers en charges pueden distorsionar los resultados.\n')
    f.write('- Las proporciones train/test se mantienen por bins.\n')

with open(os.path.join(OUTPUT_DIR, 'correlations_with_target.txt'), 'w', encoding='utf-8') as f:
    corr_target = corr['charges'].sort_values(ascending=False)
    f.write(corr_target.to_string())

with open(os.path.join(OUTPUT_DIR, 'summary.txt'), 'w', encoding='utf-8') as f:
    f.write(f'Shape original: {df.shape}\n')
    f.write(f'Shape después filtro IQR: {df_filtered.shape}\n')
    f.write(f'Train shape: {train_df.shape}\n')
    f.write(f'Test shape: {test_df.shape}\n')

## 11. Siguientes pasos
- Puedes usar los archivos generados para construir modelos predictivos (regresión, clasificación, etc).
- Revisa `output/figs/` para las visualizaciones.
- Los splits están listos para entrenar y validar modelos.